In [2]:
%matplotlib inline
import fleks,yt
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.cm import ScalarMappable
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
import numpy as np
import os
import re
from utilities import get_ticks
from matplotlib import cm, ticker
from os import listdir
from os.path import isfile, join
import glob
from PIL import Image
from natsort import natsorted
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import interpolate

# Save frame slices for making movies

In [66]:
#########################

# High-level user input
dir = "/Users/atcushen/Documents/MercuryModelling/runs/nightside_v1_run3/ta-5/"    # Directory with data 
start_time = 55                                                # What is the start time of the dataset?
t_bound = [55,77]                                               # Start and stop times for the GIF
dt = 0.2                                                     # What is the timestep between files?

'''
Choose plot type from existing presets;

"pic_E_y":  flow vectors and duskward convective field
Recommended settings: (xy)
xlims=[-2.2,0.2]
ylims=[-1.2,1.2]
shape=[160,160]

"pic_dipolarization_zoom": bz and field lines to show particular dipolarization event in detail
Recommended settings: (xz)
xlims=[-2.2,0.2]
ylims=[-1.2,1.2]
shape=[160,160] 

"pic_currentsheet": eletric field and Bz in current sheet
Recommended settings: (xz)
xlims=[-3.5,0.2]
ylims=[-1.2,1.2]
shape=[160,160] 

"bz1": perturbation magnetic field in near-planet current sheet (xy)
xlims=[-2.4,-0.4]
ylims=[-1,1]
shape=[200,200] 

"pressure_anisotropy": plot of pressure difference between perpendicular and parallel
xlims=[-2.4,-0.4]
ylims=[-1,1]
shape=[200,200] 

# "pic_currentsheet2": just bz in the current sheet (xy)
# xlims=[-2.2,0.2]
# ylims=[-1.2,1.2]
# shape=[160,160] 
'''

plot_preset = "pic_currentsheet2"
slice = "xy"                                                   # Set plane of slice ("xy" or "xz") or "sphere" for particle psd
data_type = "cut" # "cut" or "fluid"
z_plane = 0.125

# Grid settings / location of particle sphere
xlims=[-1.8,-0.8]
ylims=[-0.7,0.3]
shape=[200,200] 
loc = [-1.0,0,0.2] # location of particle sphere
a = 0.1 # radius of particle sphere

#########################

# First, get list of files in directory
all_files = [os.path.join(dir, o) for o in os.listdir(dir) 
                    if os.path.isdir(os.path.join(dir,o))]

# Then use regex to pick out only the files we want 
files=[]
for file in all_files:
    if data_type=="fluid":
        match = re.search(str(dir+"3d_fluid_region0_.*"),file)
        if match != None:
            files.append(file)
    elif data_type=="cut":
        match = re.search(str(dir+"cut_particle_region0.*"),file)
        if match != None:
            files.append(file)
files.sort()
files = files
print(str(len(files)),"files recognized!")

# Make a dictionary with better names for the files:
named_files = {}
for i in range(len(files)):
    time = round(i*dt+start_time,3)
    named_files[time] = files[i]

# Loop over each file, pull the data, make the plots, and save as .pngs in /frames
for time in list(named_files.keys())[int((t_bound[0]-start_time)/dt):int((t_bound[1]-start_time)/dt)]: #only loop over the times within t_bound
    filename = str(named_files[time])
    print(filename)
    ds = fleks.load(filename)
    if slice=="xy":
        dc = ds.get_slice("z",z_plane)
        #print(dc)
    elif slice=="xz":
        dc = ds.get_slice("y",0.0)
    elif slice=="sphere":
        ds.add_field(("particle", "unit_one"), function=_unit_one, sampling_type='particle')
    
    # Plot!!

    # Use preset
    if plot_preset == "pic_E_y":
        fig, ax = plt.subplots(figsize=(10,10))
        inner = plt.Circle((0, 0), 0.8, color='black')
        outer = plt.Circle((0, 0), 1, color='grey')
        print("Plotting t",time)
        v = dc.evaluate_expression("Ey")
        ux = np.array(dc.evaluate_expression("uxS1").T)
        uy = np.array(dc.evaluate_expression("uyS1").T)
        vmin = v.min().v
        vmax = v.max().v
        nlevels = 200 
        levels = np.linspace(-50, 50, nlevels)
        cs = ax.contourf(dc.x.value, dc.y.value, np.array(v.T)/1e3, levels=levels, cmap='bwr')
        ticks = get_ticks(-50, 50)
        clb = fig.colorbar(cs, ax=ax,ticks=ticks)            
        clb.ax.set_yticks(ticks)    
        clb.ax.tick_params(labelsize=8)
        clb.ax.set_title('$E_y$ [mV/m]',fontsize=12)
        quiver_skip=10
        ax.quiver(dc.x.value[::quiver_skip], dc.y.value[::quiver_skip],ux[::quiver_skip,::quiver_skip], uy[::quiver_skip,::quiver_skip],color="grey")
        ax.set_aspect(1) # you may also use am.imshow(..., aspect="auto") to restore the aspect ratio
        ax.set_xlabel("X [$R_M$]")
        ax.set_ylabel("Y [$R_M$]")
        ax.add_patch(outer)
        ax.add_patch(inner)
        ax.set_xlim(xlims)
        ax.set_ylim(ylims)
        ax.set_title(str("Convection field at t="+str(round(time,2))+"s"))
        fig.show()

    elif plot_preset == "pic_dipolarization_zoom":
        fig, ax = plt.subplots(figsize=(10,10))
        inner = plt.Circle((0, 0), 0.8, color='black')
        outer = plt.Circle((0, 0), 1, color='grey')
        print("Plotting t",time)
        Bx = dc.evaluate_expression("Bx")
        Bz = dc.evaluate_expression("Bz")
        vmin = -100
        vmax = 100
        nlevels = 200 
        levels = np.linspace(vmin, vmax, nlevels)
        cs = ax.contourf(dc.x.value, dc.y.value, np.array(Bz.T), levels=levels, cmap='bwr')

        start_points = np.zeros([40,2])
        start_points[:,0]=np.linspace(xlims[0],xlims[1],40)
        start_points[:,1]=0.2
        
        ax.streamplot(dc.x.value,dc.y.value, np.array(Bx.T),np.array(Bz.T),color='grey',linewidth=0.5,broken_streamlines=False,arrowsize=0.5,
                      start_points=start_points)
        
        ticks = get_ticks(vmin, vmax)
        clb = fig.colorbar(cs, ax=ax,ticks=ticks)            
        clb.ax.set_yticks(ticks)    
        clb.ax.tick_params(labelsize=8)
        clb.ax.set_title('$B_z$ [nT]',fontsize=12)
        ax.set_aspect(1) # you may also use am.imshow(..., aspect="auto") to restore the aspect ratio
        ax.set_xlabel("X [$R_M$]")
        ax.set_ylabel("Z [$R_M$]")
        ax.add_patch(outer)
        ax.add_patch(inner)
        ax.set_xlim(xlims)
        ax.set_ylim(ylims)
        ax.set_title(str("$B_z$ at t="+str(round(time,2))+"s"))
        fig.show()

    elif plot_preset == "pic_currentsheet":
        fig, ax = plt.subplots(ncols=2,figsize=(25,10))
        inner = plt.Circle((0, 0), 0.8, color='black')
        outer = plt.Circle((0, 0), 1, color='grey')

        #plot v
        v = np.sqrt(dc.evaluate_expression("uxS1")**2 + dc.evaluate_expression("uyS1")**2)
        ux = np.array(dc.evaluate_expression("uxS1").T)
        uy = np.array(dc.evaluate_expression("uyS1").T)
        vmin = v.min().v
        vmax = v.max().v
        nlevels = 200 
        levels = np.logspace(0, 3, nlevels)
        cs = ax[0].contourf(dc.x.value, dc.y.value, np.array(v.T),levels=levels, cmap='rainbow',locator=ticker.LogLocator())
        clb0 = fig.colorbar(cs, ax=ax[0])                   
        clb0.ax.tick_params(labelsize=8)
        clb0.ax.set_title('$V$ [R$_M$/s]',fontsize=12)
        quiver_skip=10
        ax[0].quiver(dc.x.value[::quiver_skip], dc.y.value[::quiver_skip],ux[::quiver_skip,::quiver_skip], uy[::quiver_skip,::quiver_skip],color="grey")
        ax[0].set_aspect(1) # you may also use am.imshow(..., aspect="auto") to restore the aspect ratio
        ax[0].set_xlabel("X [$R_M$]")
        ax[0].set_ylabel("Y [$R_M$]")
        ax[0].add_patch(outer)
        ax[0].add_patch(inner)
        ax[0].set_xlim(xlims)
        ax[0].set_ylim(ylims)
        ax[0].set_title(str("Convection field at t="+str(round(time,2))+"s"))

        #plot B
        inner = plt.Circle((0, 0), 0.8, color='black')
        outer = plt.Circle((0, 0), 1, color='grey')
        Bx = dc.evaluate_expression("Bx")
        Bz = dc.evaluate_expression("Bz")
        vmin = -100
        vmax = 100
        nlevels = 200 
        levels = np.linspace(vmin, vmax, nlevels)
        cs = ax[1].contourf(dc.x.value, dc.y.value, np.array(Bz.T), levels=levels, cmap='bwr')
        
        ticks = get_ticks(vmin, vmax)
        clb1 = fig.colorbar(cs, ax=ax[1],ticks=ticks)            
        clb1.ax.set_yticks(ticks)    
        clb1.ax.tick_params(labelsize=8)
        clb1.ax.set_title('$B_z$ [nT]',fontsize=12)
        ax[1].set_aspect(1) # you may also use am.imshow(..., aspect="auto") to restore the aspect ratio
        ax[1].set_xlabel("X [$R_M$]")
        ax[1].set_ylabel("Y [$R_M$]")
        ax[1].add_patch(outer)
        ax[1].add_patch(inner)
        ax[1].set_xlim(xlims)
        ax[1].set_ylim(ylims)
        ax[1].set_title(str("$B_z$ at t="+str(round(time,2))+"s"))

    elif plot_preset == "particles_uxuy":
        # broken for many reasons, and the particle data is featureless anyway.
        sp = ds.sphere(loc,a)
        plot = ds.plot_phase_region(sp, 'p_ux', 'p_uz', z_field,
                            unit_type="planet", x_bins=64, y_bins=64,domain_size=(-2550, 2550, -2550, 2550))
        #plot.set_colorbar_label(plot.fields[0], "Ion count")
        for var_name in plot.profile.field_data: 
            val = plot.profile.field_data[var_name] 
    
        x = plot.profile.x
        y = plot.profile.y

        fig, ax = plt.subplots(figsize=(10,10))
        im = ax.contourf(x,y,val,20,vmin = 0,vmax=700)
        im.set_clim(0,700)
        clb=fig.colorbar(im)

    elif plot_preset == "pic_currentsheet2":
        fig, ax = plt.subplots(figsize=(10,10))
        inner = plt.Circle((0, 0), 0.8, color='black')
        outer = plt.Circle((0, 0), 1, color='grey')
        print("Plotting t",time)
        Bz = dc.evaluate_expression("Bz")
        vmin = -100
        vmax = 100
        nlevels = 200 
        levels = np.linspace(vmin, vmax, nlevels)
        cs = ax.contourf(dc.x.value, dc.y.value, np.array(Bz.T), levels=levels, cmap='bwr')

        # Add minor ticks for grid
        x_major_ticks = np.arange(xlims[0], xlims[1], 0.2)
        x_minor_ticks = np.arange(xlims[0], xlims[1], 0.05)
        y_major_ticks = np.arange(ylims[0], ylims[1], 0.2)
        y_minor_ticks = np.arange(ylims[0], ylims[1], 0.05)

        ax.set_xticks(x_major_ticks)
        ax.set_xticks(x_minor_ticks, minor=True)
        ax.set_yticks(y_major_ticks)
        ax.set_yticks(y_minor_ticks, minor=True)

        ax.grid(which='both')
        ax.grid(which='minor', alpha=0.2)
        ax.grid(which='major', alpha=0.5)
        
        ticks = get_ticks(vmin, vmax)
        clb = fig.colorbar(cs, ax=ax,ticks=ticks)            
        clb.ax.set_yticks(ticks)    
        clb.ax.tick_params(labelsize=8)
        clb.ax.set_title('$B_z$ [nT]',fontsize=12)
        ax.set_aspect(1) # you may also use am.imshow(..., aspect="auto") to restore the aspect ratio
        ax.set_xlabel("X [$R_M$]")
        ax.set_ylabel("Y [$R_M$]")
        ax.add_patch(outer)
        ax.add_patch(inner)
        ax.set_xlim(xlims)
        ax.set_ylim(ylims)
        ax.set_title(str("$B_z$ at t="+str(round(time,2))+"s"))
        fig.show()

    elif plot_preset == "bz1":
        fig, ax = plt.subplots(figsize=(10,10))
        inner = plt.Circle((0, 0), 0.8, color='black')
        outer = plt.Circle((0, 0), 1, color='grey')
        print("Plotting t",time)

        # Extract Bz and remove units
        Bz_temp = dc.evaluate_expression("Bz")
        Bz = np.zeros(Bz_temp.shape)
        Bz[:,:] = Bz_temp[:,:]

        # Define meshgrids
        xx,yy = np.meshgrid(dc.x.value, dc.y.value)

        #Compute Bz1 
        #First compute dipole field strength, from https://ccmc.gsfc.nasa.gov/static/files/Dipole.pdf
        Bz_dip = - 200.9 * (3*z_plane**2 - (xx**2+yy**2+z_plane**2))/((xx**2+yy**2+z_plane**2)**(5/2))
        # Then subtract this from modeled bz
        Bz1 = Bz.T - Bz_dip
        
        vmin = -100#-100
        vmax = 0
        nlevels = 200 
        levels = np.linspace(vmin, vmax, nlevels)
        
        cs = ax.contourf(dc.x.value, dc.y.value, np.array(Bz1), cmap = 'viridis',levels=levels) #,cmap='bwr')

        # Add minor ticks for grid
        x_major_ticks = np.arange(xlims[0], xlims[1], 0.2)
        x_minor_ticks = np.arange(xlims[0], xlims[1], 0.05)
        y_major_ticks = np.arange(ylims[0], ylims[1], 0.2)
        y_minor_ticks = np.arange(ylims[0], ylims[1], 0.05)

        ax.set_xticks(x_major_ticks)
        ax.set_xticks(x_minor_ticks, minor=True)
        ax.set_yticks(y_major_ticks)
        ax.set_yticks(y_minor_ticks, minor=True)

        ax.grid(which='both')
        ax.grid(which='minor', alpha=0.2)
        ax.grid(which='major', alpha=0.5)
        
        ticks = get_ticks(vmin, vmax)
        clb = fig.colorbar(cs, ax=ax,ticks=ticks)            
        clb.ax.set_yticks(ticks)    
        clb.ax.tick_params(labelsize=8)
        clb.ax.set_title('$B_z$ [nT]',fontsize=12)
        ax.set_aspect(1) # you may also use am.imshow(..., aspect="auto") to restore the aspect ratio
        ax.set_xlabel("X [$R_M$]")
        ax.set_ylabel("Y [$R_M$]")
        ax.add_patch(outer)
        ax.add_patch(inner)
        ax.set_xlim(xlims)
        ax.set_ylim(ylims)
        ax.set_title(str("$B_z$ at t="+str(round(time,2))+"s"+"   z = "+str(z_plane)))
        fig.show()

    elif plot_preset == "pressure_anisotropy":
        fig, ax = plt.subplots(figsize=(10,10))
        inner = plt.Circle((0, 0), 0.8, color='black')
        outer = plt.Circle((0, 0), 1, color='grey')
        print("Plotting t",time)

        # Extract pressure terms and remove units
        pxxS1_temp = dc.evaluate_expression("pxxS1")
        pxyS1_temp = dc.evaluate_expression("pxyS1")
        pxzS1_temp = dc.evaluate_expression("pxzS1")
        pyyS1_temp = dc.evaluate_expression("pyyS1")
        pyzS1_temp = dc.evaluate_expression("pyzS1")
        pzzS1_temp = dc.evaluate_expression("pzzS1")
        pxxS1 = np.zeros(pxxS1_temp.T.shape)
        pxyS1 = np.zeros(pxyS1_temp.T.shape)
        pxzS1 = np.zeros(pxzS1_temp.T.shape)
        pyyS1 = np.zeros(pyyS1_temp.T.shape)
        pyzS1 = np.zeros(pyzS1_temp.T.shape)
        pzzS1 = np.zeros(pzzS1_temp.T.shape)
        pxxS1[:,:] = pxxS1_temp[:,:].T
        pxyS1[:,:] = pxyS1_temp[:,:].T
        pxzS1[:,:] = pxyS1_temp[:,:].T
        pyyS1[:,:] = pyyS1_temp[:,:].T
        pyzS1[:,:] = pyzS1_temp[:,:].T
        pzzS1[:,:] = pzzS1_temp[:,:].T

        # Extract magnetic terms and remove units
        Bx_temp = dc.evaluate_expression("Bx")
        By_temp = dc.evaluate_expression("By")
        Bz_temp = dc.evaluate_expression("Bz")
        Bx = np.zeros(Bx_temp.T.shape)
        By = np.zeros(By_temp.T.shape)
        Bz = np.zeros(Bz_temp.T.shape)
        Bx[:,:] = Bx_temp[:,:].T
        By[:,:] = By_temp[:,:].T
        Bz[:,:] = Bz_temp[:,:].T

        # Define meshgrids
        xx,yy = np.meshgrid(dc.x.value, dc.y.value)

        # Compute magnetic unit vector
        B_mag = np.sqrt(Bx**2+By**2+Bz**2)
        bx,by,bz = 1/B_mag * [Bx,By,Bz]

        # Compute coordinate transformation matrix, A
        # The three columns of this matrix correspond to the first perpendicular component, the second, and the parallel component, respectively
        Axx = np.sqrt(1-bx**2)
        Axy = 0
        Axz = bx
        Ayx = -bx*by/np.sqrt(1-bx**2)
        Ayy = bz/np.sqrt(bz**2+by**2)
        Ayz = by
        Azx = -bx*bz/np.sqrt(1-bx**2)
        Azy = -by/np.sqrt(bz**2+by**2)
        Azz = bz

        # Transfrom the pressure tensor from pS1 (in cartesian) to P (in perp / perp / parallel coords): P = A x pS1
        # Note that pS1 is symmetric, so e.g. pxyS1 = pyxS1
        Pxx = Axx*pxxS1 + Axy*pxyS1 + Axz*pxzS1
        Pxy = Axx*pxyS1 + Axy*pyyS1 + Axz*pyzS1
        Pxz = Axx*pxzS1 + Axy*pyzS1 + Axz*pzzS1
        Pyx = Ayx*pxxS1 + Ayy*pxyS1 + Ayz*pxzS1
        Pyy = Ayx*pxyS1 + Ayy*pyyS1 + Ayz*pyzS1
        Pyz = Ayx*pxzS1 + Ayy*pyzS1 + Ayz*pzzS1
        Pzx = Azx*pxxS1 + Azy*pxyS1 + Azz*pxzS1
        Pzy = Azx*pxyS1 + Azy*pyyS1 + Azz*pyzS1
        Pzz = Azx*pxzS1 + Azy*pyzS1 + Azz*pzzS1

        # For now, we are interested in the difference parallel (Pzz) and parallel (Pxx and Pyy)
        P_anisotropy = Pzz - Pxx - Pyy / np.sqrt(Pzz**2+Pxx**2+Pyy**2)

        # Plot
        vmin = -5
        vmax = 5
        nlevels = 200 
        levels = np.linspace(vmin, vmax, nlevels)
        
        cs = ax.contourf(dc.x.value, dc.y.value, np.array(P_anisotropy), levels=levels, cmap='bwr')

        # Add minor ticks for grid
        x_major_ticks = np.arange(xlims[0], xlims[1], 0.2)
        x_minor_ticks = np.arange(xlims[0], xlims[1], 0.05)
        y_major_ticks = np.arange(ylims[0], ylims[1], 0.2)
        y_minor_ticks = np.arange(ylims[0], ylims[1], 0.05)

        ax.set_xticks(x_major_ticks)
        ax.set_xticks(x_minor_ticks, minor=True)
        ax.set_yticks(y_major_ticks)
        ax.set_yticks(y_minor_ticks, minor=True)

        ax.grid(which='both')
        ax.grid(which='minor', alpha=0.2)
        ax.grid(which='major', alpha=0.5)
        
        ticks = get_ticks(vmin, vmax)
        clb = fig.colorbar(cs, ax=ax,ticks=ticks)            
        clb.ax.set_yticks(ticks)    
        clb.ax.tick_params(labelsize=8)
        clb.ax.set_title('Normalized diff',fontsize=12)
        ax.set_aspect(1) # you may also use am.imshow(..., aspect="auto") to restore the aspect ratio
        ax.set_xlabel("X [$R_M$]")
        ax.set_ylabel("Y [$R_M$]")
        ax.add_patch(outer)
        ax.add_patch(inner)
        ax.set_xlim(xlims)
        ax.set_ylim(ylims)
        ax.set_title(str("Perpendicular - parallel at t="+str(round(time,2))+"s"+"   z = "+str(z_plane)))
        fig.show()
        
    # Save
    plt.savefig(str("/Users/atcushen/Documents/MercuryModelling/runs/nightside_v1_run3/ta-5_plots/"+plot_preset+"_"+str(round(time,2))+'.png'),bbox_inches='tight')
    plt.close(fig)

yt : [INFO     ] 2024-07-22 15:42:56,708 Parameters: current_time              = 55.000645822431196
yt : [INFO     ] 2024-07-22 15:42:56,709 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 15:42:56,710 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 15:42:56,710 Parameters: domain_right_edge         = [1. 2. 2.]


75 files recognized!
/Users/atcushen/Documents/MercuryModelling/runs/nightside_v1_run3/ta-5/cut_particle_region0_0_t00000055_n00030455_amrex
Plotting t 55.0


/var/folders/l9/dnt9qhfd1psfsmj9t5zfr0j00000gq/T/ipykernel_17737/104369098.py:276: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()
yt : [INFO     ] 2024-07-22 15:42:58,533 Parameters: current_time              = 55.20049731286974
yt : [INFO     ] 2024-07-22 15:42:58,534 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 15:42:58,534 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 15:42:58,535 Parameters: domain_right_edge         = [1. 2. 2.]


/Users/atcushen/Documents/MercuryModelling/runs/nightside_v1_run3/ta-5/cut_particle_region0_0_t00000055_n00030579_amrex
Plotting t 55.2


yt : [INFO     ] 2024-07-22 15:43:00,439 Parameters: current_time              = 55.40075914152486
yt : [INFO     ] 2024-07-22 15:43:00,439 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 15:43:00,440 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 15:43:00,440 Parameters: domain_right_edge         = [1. 2. 2.]


...

yt : [INFO     ] 2024-07-22 15:44:43,721 Parameters: current_time              = 66.80229931444407
yt : [INFO     ] 2024-07-22 15:44:43,722 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 15:44:43,723 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 15:44:43,723 Parameters: domain_right_edge         = [1. 2. 2.]


/Users/atcushen/Documents/MercuryModelling/runs/nightside_v1_run3/ta-5/cut_particle_region0_0_t00000106_n00037628_amrex
Plotting t 66.8


yt : [INFO     ] 2024-07-22 15:44:45,445 Parameters: current_time              = 67.00037018794269
yt : [INFO     ] 2024-07-22 15:44:45,446 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 15:44:45,446 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 15:44:45,447 Parameters: domain_right_edge         = [1. 2. 2.]


/Users/atcushen/Documents/MercuryModelling/runs/nightside_v1_run3/ta-5/cut_particle_region0_0_t00000107_n00037713_amrex


KeyboardInterrupt: 

# Particle data plotting


In [3]:
####### START USER INPUT ###########
folder = "/Users/atcushen/Documents/MercuryModelling/runs/nightside_v1_run3/ta-5"
file = "cut_particle_region0_0_t00000031_n00016670_amrex/"
loc=[-1.4,-0.2,0.125]  # Location of sphere to sample particles in
radius=(0.05)  # Size of sphere
plot_preset = "v_para-v_perp"
multi = True # Loop through files and save multiple panels?
start_time = 55            # What is the earliest time saved in this directory?
t_bound = [55,65]                # Start and stop times to loop through
dt = 0.2  # time step between files
make_gif = False    # whether to save a gif. Not working at the moment
####### END USER INPUT ###########
dir = str(folder+"/")
""" 
PRESETS:
vx_vy-vx_vz: plot of raw particle velocity distribution at loc, using standard values for controls. 

v_para-v_perp: plot of particle velocity distribution at loc, in parallel / perpendicular coords

"""

# Make a dictionary with better names for the files:
#named_files = {}
#for i in range(len(files)):
#    time = round(i*dt+start_time,3)
#    named_files[time] = files[i]

def get_directories(dir, key=".*cut_particle_region0.*"):
    # For a directory "dir", return a list of all files which match the regex expression "key"
    all_files = [ f.name for f in os.scandir(dir) if f.is_dir() ]
    files=[]
    for file in all_files:
        match = re.search(key,file)
        if match != None:
            files.append(file)
    files.sort()
    # Now give them the appropriate name for their time
    named_files = {}
    for i in range(len(files)):
        time = round(i*dt+start_time,3)
        named_files[time] = files[i]
    # Now cut the list down to files inside t_bound
    reduced_files = {}
    for time in list(named_files.keys())[int((t_bound[0]-start_time)/dt):int((t_bound[1]-start_time)/dt)]: #only loop over the times within t_bound
        reduced_files[time] = str(named_files[time])
    return reduced_files

def _unit_one(field, data):
    res = np.zeros(data[('particle', 'p_w')].shape)
    res[:] = 1
    return res

def get_B_at_loc(loc,data):
    # Input coordinates and ds of file to get B at that location
    dc = data.get_slice("z",loc[2])

    # Pull the coord data
    x,y = dc.x.value, dc.y.value
    
    # Find the cell indices surrounding loc
    xi_min = np.where(x < loc[0])[0][-1]
    yi_min = np.where(y < loc[1])[0][-1]
    
    # Define the trimmed meshgrids
    xx,yy = np.meshgrid(x[xi_min:xi_min+2],y[yi_min:yi_min+2])
    
    # Pull only the relevant mag data
    Bx = np.array(dc.evaluate_expression("Bx").T)[xi_min:xi_min+2,yi_min:yi_min+2]
    By = np.array(dc.evaluate_expression("By").T)[xi_min:xi_min+2,yi_min:yi_min+2]
    Bz = np.array(dc.evaluate_expression("Bz").T)[xi_min:xi_min+2,yi_min:yi_min+2]

    # Do a little interpolation: B = B[0,0] + dB/dX dx + dB/dY dy
    Bx_loc = Bx[0,0] + (Bx[0,1]-Bx[0,0])/(xx[0,1]-xx[0,0])*(loc[0]-xx[0,0]) + (Bx[1,0]-Bx[0,0])/(yy[1,0]-yy[0,0])*(loc[1]-yy[0,0])
    By_loc = By[0,0] + (By[0,1]-By[0,0])/(xx[0,1]-xx[0,0])*(loc[0]-xx[0,0]) + (By[1,0]-By[0,0])/(yy[1,0]-yy[0,0])*(loc[1]-yy[0,0])
    Bz_loc = Bz[0,0] + (Bz[0,1]-Bz[0,0])/(xx[0,1]-xx[0,0])*(loc[0]-xx[0,0]) + (Bz[1,0]-Bz[0,0])/(yy[1,0]-yy[0,0])*(loc[1]-yy[0,0])

    return Bx_loc,By_loc,Bz_loc

def get_B_at_loc_from_fluid(file,loc):
    # Hand the file name of cut data being used, for which the corresponding fluid data will be opened to read B
    # This was created as a workaround to get_B_at_loc, which started to fail since it read B as being 0. Couldn't work out why,
    # so now we rely on also downloading the fluid data
    # First, extract the iteration number from the filename
    iteration = file[-15:]

    # Then, pull the matching 3d fluid file
    fluid_file = get_directories(dir, key=str(".*3d_fluid.*"+iteration))
    fluid_file = fluid_file[list(fluid_file.keys())[0]]

    # Load the data and slice
    fluid_ds = fleks.load(str(dir+fluid_file))
    dc = fluid_ds.get_slice("z",loc[2])

    # Pull the coord data
    x,y = dc.x.value, dc.y.value
    
    # Find the cell indices surrounding loc
    xi_min = np.where(x < loc[0])[0][-1]
    yi_min = np.where(y < loc[1])[0][-1]
    
    # Define the trimmed meshgrids
    xx,yy = np.meshgrid(x[xi_min:xi_min+2],y[yi_min:yi_min+2])
    
    # Pull only the relevant mag data
    Bx = np.array(dc.evaluate_expression("Bx"))[xi_min:xi_min+2,yi_min:yi_min+2].T
    By = np.array(dc.evaluate_expression("By"))[xi_min:xi_min+2,yi_min:yi_min+2].T
    Bz = np.array(dc.evaluate_expression("Bz"))[xi_min:xi_min+2,yi_min:yi_min+2].T

    # Do a little interpolation: B = B[0,0] + dB/dX dx + dB/dY dy
    Bx_loc = Bx[0,0] + (Bx[0,1]-Bx[0,0])/(xx[0,1]-xx[0,0])*(loc[0]-xx[0,0]) + (Bx[1,0]-Bx[0,0])/(yy[1,0]-yy[0,0])*(loc[1]-yy[0,0])
    By_loc = By[0,0] + (By[0,1]-By[0,0])/(xx[0,1]-xx[0,0])*(loc[0]-xx[0,0]) + (By[1,0]-By[0,0])/(yy[1,0]-yy[0,0])*(loc[1]-yy[0,0])
    Bz_loc = Bz[0,0] + (Bz[0,1]-Bz[0,0])/(xx[0,1]-xx[0,0])*(loc[0]-xx[0,0]) + (Bz[1,0]-Bz[0,0])/(yy[1,0]-yy[0,0])*(loc[1]-yy[0,0])

    return Bx_loc,By_loc,Bz_loc

def get_particle_np(file, x_axis='p_ux', y_axis='p_uy', bins=64, v_lim = 2500):
    # Get the particle distribution in numpy format.
    # File is the directory of the amrex folder (e.g. /Users/*_amrex/"
    # x_axis and y_axis are what quantities to extract e.g. 'p_x','p_y'
    # Bins is how many bins to divide into
    # v_lim is the maximum velocity to extract (in km/s)
    # Outputs the meshgrids of velocities (in km/s), and the number of particles in each of those grid cells

    # Load the file
    ds = fleks.load(file)

    if x_axis == 'p_ux': # just check what the x_axis is to determine which mode we are in; this is temporary
        ds.add_field(("particle", "unit_one"), function=_unit_one, sampling_type='particle')
        sp = ds.sphere(loc,radius)
        z_field = 'unit_one'
        plot = ds.plot_phase_region(sp, x_axis, y_axis, z_field, unit_type="planet", 
                                        x_bins=bins, y_bins=bins, domain_size=(-v_lim, v_lim, -v_lim, v_lim))
    
        for var_name in plot.profile.field_data: 
            counts_temp = plot.profile.field_data[var_name].T # take the transpose, which seems to be necessary. Bit worried about that.

    elif x_axis == 'v_para': 
        # Do a little interpolation: B = B[0,0] + dB/dX dx + dB/dY dy
        Bx_loc,By_loc,Bz_loc = get_B_at_loc_from_fluid(file,loc)
        
        # Define magnetic field unit vectors at loc
        bhat = [Bx_loc,By_loc,Bz_loc]/np.sqrt(Bx_loc**2+By_loc**2+Bz_loc**2)
        
        # Sometimes B is identically 0 for some weird reason... so we get nan for bhat. Not sure how this is possible.
        bhat[np.isnan(bhat)] = 0
        bx,by,bz = bhat
        
        # For the particle data, we must create user-defined fields for the parallel and perpedicular components. See notebook for handwritten derivation.
        def _vel_para(field, data): # This is basically v_para = v . b
            res = bx*data[('particle', 'p_ux')] + by*data[('particle', 'p_uy')] + bz*data[('particle', 'p_uz')]        
            return res
        def _vel_perp(field, data): # This is v_perp = v - v_para
            res = (( data[('particle', 'p_ux')] - bx * ( bx*data[('particle', 'p_ux')]) + by*data[('particle', 'p_uy')] + bz*data[('particle', 'p_uz')] )**2 +
                   ( data[('particle', 'p_uy')] - by * ( bx*data[('particle', 'p_ux')]) + by*data[('particle', 'p_uy')] + bz*data[('particle', 'p_uz')] )**2 +
                   ( data[('particle', 'p_uz')] - bz * ( bx*data[('particle', 'p_ux')]) + by*data[('particle', 'p_uy')] + bz*data[('particle', 'p_uz')] )**2 )**0.5        
            return res
        
        # Add the fields to the dataset
        vpara_name = ds.pvar("vel_para")
        vperp_name = ds.pvar("vel_perp")
        ds.add_field(vpara_name, units="code_velocity", function=_vel_para, sampling_type='particle')
        ds.add_field(vperp_name, units="code_velocity", function=_vel_perp, sampling_type='particle')

        # Generate plot 
        ds.add_field(("particle", "unit_one"), function=_unit_one, sampling_type='particle')
        sp = ds.sphere(loc,radius)
        x_field = vpara_name
        y_field = vperp_name
        z_field = ("particle", "unit_one")#ds.pvar('p_w')
        logs = {x_field: False, y_field: False}
        profile = yt.create_profile(data_source=sp, bin_fields=[x_field,y_field], fields=z_field, n_bins=[64,64], weight_field=None, logs=logs)
        plot = yt.PhasePlot.from_profile(profile)
        plot.set_unit(x_field, "km/s")
        plot.set_unit(y_field, "km/s")
        plot.set_unit(z_field, "amu")
        for var_name in plot.profile.field_data: 
            counts_temp = plot.profile.field_data[var_name].T

    # Convert to np. Define output arrays without units
    x_temp = plot.profile.x
    y_temp = plot.profile.y
    x=np.zeros(x_temp.shape)
    y=np.zeros(y_temp.shape)
    counts=np.zeros(counts_temp.shape)
    x[:]=x_temp[:]
    y[:]=y_temp[:]
    counts[:,:]=counts_temp[:,:]
    xx,yy=np.meshgrid(x,y)

    return xx,yy,counts

def generate_gif(folder, key=str(plot_preset+"*")):
    #read all the .png files in directory "frames" 
    print(folder)
    print(key)
    gif_files = glob.glob(str(folder+"/"+key))
    print(gif_files)
    image_array = []
    gif_files = natsorted(gif_files)
    
    for my_file in gif_files:
        image = Image.open(gif_files)
        image_array.append(image)
        
    # Create the figure and axes objects
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Set the initial image
    im = ax.imshow(image_array[0], animated=True)
    plt.axis('off')
    
    def update(i):
        im.set_array(image_array[i])
        return im, 
    
    # Create the animation object
    animation_fig = animation.FuncAnimation(fig, update, frames=len(image_array), interval=200, blit=True,repeat_delay=10,)
    
    animation_fig.save(str(folder+"_plots/"+"animated.gif"),dpi=500)
    plt.close(fig)

def particle_plots(mode):
    # Generates the desired plot for the given preset.

    if mode == "vx_vy-vx_vz":
        # Define the files to read
        if multi:
            print("Plotting all files in dir")
            files = get_directories(dir)
        else:
            files = {str(round(start_time,3)):str(file)}

        # Loop over the identified files
        for time in list(files.keys()): 
            print("Plotting t =",time)
            file = str(files[time])
        
            # Get data
            data_file = dir+file
            xx,yy,counts_xy = get_particle_np(data_file,'p_ux','p_uy')
            xx,zz,counts_xz = get_particle_np(data_file,'p_ux','p_uz')
    
            # Set colorbar limits
            cmin = 1
            cmax = 100#max(counts_xy.max(),counts_xz.max())
    
            # Plot
            fig, axs = plt.subplots(ncols=2,figsize=(13,5))
            im0=axs[0].imshow(counts_xy, extent=[xx.min(), xx.max(), yy.min(), yy.max()],origin="lower",norm=LogNorm(vmin=cmin,vmax=cmax))
            im1=axs[1].imshow(counts_xz, extent=[xx.min(), xx.max(), zz.min(), zz.max()],origin="lower",norm=LogNorm(vmin=cmin,vmax=cmax))
    
            # Add labels
            axs[0].set_xlabel("$v_x$ [km/s]")
            axs[0].set_ylabel("$v_y$ [km/s]")
            axs[1].set_xlabel("$v_x$ [km/s]")
            axs[1].set_ylabel("$v_z$ [km/s]")
            axs[0].set_title(str("Velocity distribution  r="+str(loc)+"   t="+str(round(time,2))+"s"))
    
            # Add grid
            axs[0].grid(alpha=0.5)
            axs[1].grid(alpha=0.5)
    
            # Add colorbar
            fig.subplots_adjust(right=0.84)
            cbar_ax = fig.add_axes([0.85, 0.15, 0.01, 0.7])
            fig.colorbar(im1, cax=cbar_ax, label = "Counts")
    
            # Save
            fig.savefig(str(folder+"_plots/"+plot_preset+"_"+str(loc)+"_"+str(round(time,2))+'.png'),bbox_inches='tight')

            if multi:
                plt.close()

            else:
                plt.show()
        
        if make_gif:
            generate_gif(folder)

    if mode == "v_para-v_perp":
        # Define the files to read
        if multi:
            print("Plotting all files in dir")
            files = get_directories(dir)
        else:
            files = {str(round(start_time,3)):str(file)}

        # Loop over the identified files
        for time in list(files.keys()): 
            print("Plotting t =",time)
            file = str(files[time])

            # Get data
            data_file = dir+file
            xx,yy,counts = get_particle_np(data_file,'v_para','v_perp')
            
            # Get B field data
            ds = fleks.load(data_file)
            Bx_loc,By_loc,Bz_loc = get_B_at_loc_from_fluid(file,loc)
            
            # Set colorbar limits
            cmin = 1
            cmax = 100
    
            # Plot
            fig, ax = plt.subplots(ncols=1,figsize=(7,7))
            im=ax.imshow(counts.T, extent=[yy.min(), yy.max(), xx.min(), xx.max()],origin="lower",norm=LogNorm(vmin=cmin,vmax=cmax))
    
            # Add labels
            ax.set_ylabel("$v_{para}$ [km/s]")
            ax.set_xlabel("$v_{perp}$ [km/s]")
            ax.set_title(str("Velocity distribution  r="+str(loc)+"   t="+str(round(time,2))+"s \n Local B: ("+str(round(Bx_loc,3))+", "+str(round(By_loc,3))+", "+str(round(Bz_loc,3))+")"))
            ax.set_ylim(-1600,1600)
            ax.set_xlim(0,3000)
    
            # Add grid
            ax.grid(alpha=0.5)
            ax.grid(alpha=0.5)
    
            # Add colorbar
            #fig.subplots_adjust(right=0.84)
            #cbar_ax = fig.add_axes([0.85, 0.15, 0.01, 0.7])
            fig.colorbar(im, label = "Counts")
    
            # Save
            fig.savefig(str(folder+"_plots/"+plot_preset+"_"+str(loc)+"_"+str(round(time,2))+'.png'),bbox_inches='tight')

            if multi:
                plt.close()

            else:
                plt.show()
        
        if make_gif:
            generate_gif(folder)
# Run
particle_plots(plot_preset)

yt : [INFO     ] 2024-07-22 16:08:46,845 Parameters: current_time              = 55.000645822431196
yt : [INFO     ] 2024-07-22 16:08:46,846 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 16:08:46,847 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 16:08:46,847 Parameters: domain_right_edge         = [1. 2. 2.]
yt : [INFO     ] 2024-07-22 16:08:46,868 Parameters: current_time              = 55.000645822431196
yt : [INFO     ] 2024-07-22 16:08:46,868 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 16:08:46,869 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 16:08:46,870 Parameters: domain_right_edge         = [1. 2. 2.]


Plotting all files in dir
Plotting t = 55.0


yt : [WARNING  ] 2024-07-22 16:08:48,519 Field ('particle', 'unit_one') was added without specifying units or dimensions, auto setting units to 
yt : [INFO     ] 2024-07-22 16:08:48,621 Parameters: current_time              = 55.000645822431196
yt : [INFO     ] 2024-07-22 16:08:48,622 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 16:08:48,623 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 16:08:48,630 Parameters: domain_right_edge         = [1. 2. 2.]
yt : [INFO     ] 2024-07-22 16:08:48,688 Parameters: current_time              = 55.000645822431196
yt : [INFO     ] 2024-07-22 16:08:48,691 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 16:08:48,693 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 16:08:48,695 Parameters: domain_right_edge         = [1. 2. 2.]
yt : [INFO     ] 2024-07-22 16:08:49,491 Parameters: current_time              = 55.2004973

Plotting t = 55.2


yt : [WARNING  ] 2024-07-22 16:08:51,147 Field ('particle', 'unit_one') was added without specifying units or dimensions, auto setting units to 
yt : [INFO     ] 2024-07-22 16:08:51,283 Parameters: current_time              = 55.20049731286974
yt : [INFO     ] 2024-07-22 16:08:51,284 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 16:08:51,284 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 16:08:51,285 Parameters: domain_right_edge         = [1. 2. 2.]
yt : [INFO     ] 2024-07-22 16:08:51,303 Parameters: current_time              = 55.20049731286974
yt : [INFO     ] 2024-07-22 16:08:51,304 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 16:08:51,305 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 16:08:51,306 Parameters: domain_right_edge         = [1. 2. 2.]
yt : [INFO     ] 2024-07-22 16:08:52,023 Parameters: current_time              = 55.400759141

...

yt : [WARNING  ] 2024-07-22 16:10:53,065 Field ('particle', 'unit_one') was added without specifying units or dimensions, auto setting units to 
yt : [INFO     ] 2024-07-22 16:10:53,132 Parameters: current_time              = 64.40128991949511
yt : [INFO     ] 2024-07-22 16:10:53,132 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 16:10:53,133 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 16:10:53,134 Parameters: domain_right_edge         = [1. 2. 2.]
yt : [INFO     ] 2024-07-22 16:10:53,152 Parameters: current_time              = 64.40128991949511
yt : [INFO     ] 2024-07-22 16:10:53,152 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 16:10:53,153 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 16:10:53,153 Parameters: domain_right_edge         = [1. 2. 2.]
yt : [INFO     ] 2024-07-22 16:10:53,815 Parameters: current_time              = 64.600219857

Plotting t = 64.6


yt : [WARNING  ] 2024-07-22 16:10:55,658 Field ('particle', 'unit_one') was added without specifying units or dimensions, auto setting units to 
yt : [INFO     ] 2024-07-22 16:10:55,729 Parameters: current_time              = 64.60021985749586
yt : [INFO     ] 2024-07-22 16:10:55,730 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 16:10:55,731 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 16:10:55,732 Parameters: domain_right_edge         = [1. 2. 2.]
yt : [INFO     ] 2024-07-22 16:10:55,750 Parameters: current_time              = 64.60021985749586
yt : [INFO     ] 2024-07-22 16:10:55,751 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 16:10:55,751 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 16:10:55,752 Parameters: domain_right_edge         = [1. 2. 2.]
yt : [INFO     ] 2024-07-22 16:10:56,425 Parameters: current_time              = 64.801386390

Plotting t = 64.8


yt : [WARNING  ] 2024-07-22 16:10:58,264 Field ('particle', 'unit_one') was added without specifying units or dimensions, auto setting units to 
yt : [INFO     ] 2024-07-22 16:10:58,331 Parameters: current_time              = 64.80138639009368
yt : [INFO     ] 2024-07-22 16:10:58,331 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 16:10:58,332 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 16:10:58,333 Parameters: domain_right_edge         = [1. 2. 2.]
yt : [INFO     ] 2024-07-22 16:10:58,351 Parameters: current_time              = 64.80138639009368
yt : [INFO     ] 2024-07-22 16:10:58,351 Parameters: domain_dimensions         = [320 256 256]
yt : [INFO     ] 2024-07-22 16:10:58,352 Parameters: domain_left_edge          = [-4. -2. -2.]
yt : [INFO     ] 2024-07-22 16:10:58,352 Parameters: domain_right_edge         = [1. 2. 2.]


In [5]:
dc

NameError: name 'dc' is not defined